In [2]:
# Create new features, focusing on high ROI features. 

In [5]:
# Features I already have _______________________________________
# - rewards: n, min, mode, max, intn shipping
# - n creator backed
# - title char len - change to n chars
# - blurb char len - change to n chars
# - launch time (in Unix time) - drop
# - project duration - in days
# - goal

In [6]:
# Done! _______________________
# - day of the week
# - day of the month -- 1st/15st/30st
# - time between launch and mode ship of rewards

In [7]:
# To add _______________________________________
# - body n words
# - n pictures

In [8]:
# I missed this! _________ 
# - has video
# rescrape, just check video/no video on main page. this weekend.

In [9]:
# low roi _____________________
# - faq n questions/word len
# - risks and rewards word len
# - title word len
# - blurb word len
# - readablity 
# - n misspellings

In [10]:
from bs4 import BeautifulSoup
import pprint
import pymysql as mdb
import re
import pandas as pd
import numpy as np
from unidecode import unidecode
import json
import time

In [140]:
from extract_kickstarter_features import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
con = mdb.connect('localhost','root','hobbes','kickstarter')

In [14]:
with con:
    
    cur = con.cursor()
    cur.execute("SHOW TABLES")
    
    rows = cur.fetchall()
    for row in rows:
        print row

('Creator',)
('Project_info',)
('Rewards',)
('Scraped_pages',)


In [153]:
tech_df = pd.read_pickle('tech_df.save')
tech_df.head()

,id,pledged,goal,n rewards,min reward,max reward,intn ship,n creator backed,title n chars,blurb n chars,launch time,campaign dur,days to ship,month,month day,week day,hour,campaign_dur
0,8018,90840,20000,13,1,10000,1,15,29,109,1391619643,2592000,176,2,5,2,12,30
1,8019,55833,50000,12,10,10000,1,1,33,105,1413818746,2595600,193,10,20,0,11,30.04167
2,8020,6804,5000,10,1,5000,-1,12,18,129,1361591034,5184000,67,2,22,4,22,60
3,8021,313218,5000,14,10,1000,-1,24,53,95,1344533619,2715981,83,8,9,3,13,31.43497
4,8022,74446,10000,9,9,300,1,0,22,122,1393448405,2627995,124,2,26,2,16,30.41661


In [128]:
with con:
    cur = con.cursor()
    for i, Id in enumerate(tech_df.ix[6628:,'id']): #tech_df['id']:
        print i
        primary_key = int(Id)
        cur.execute("SELECT URL FROM Project_info WHERE Id = %s",primary_key)
        rows = cur.fetchall()
        print rows[0][0]
    cur.close()


0
https://www.kickstarter.com/projects/1658481240/border-fence-simplicity?ref=category
1
https://www.kickstarter.com/projects/205288304/v-desktop-sensor-control?ref=category
2
https://www.kickstarter.com/projects/1916559806/liquid-screen-shield-a-truly-invisible-screen-prot?ref=category
3
https://www.kickstarter.com/projects/1205602507/one-day-decisions-changes-americas-justice-system?ref=category
4
https://www.kickstarter.com/projects/augustgermar/anonabox-a-tor-hardware-router?ref=category


In [155]:
with con:
    cur = con.cursor()
    for i, Id in enumerate(tech_df['id']): 
        #print i 
        if not i%500:
           print i
            
        primary_key = int(Id)
        cur.execute("SELECT Project_html FROM Scraped_pages WHERE Id = %s",primary_key)
        rows = cur.fetchall()
        project_html = rows[0][0]
        
        #print BeautifulSoup(project_html).get_text()
        
        n_links, n_figs,body_n_words = analyze_description_features(project_html)
        
        tech_df.ix[i,'n links'] = n_links
        tech_df.ix[i,'n figs'] = n_figs
        tech_df.ix[i,'body n words'] = body_n_words
        
    cur.close()

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
